In [1]:
import random
from util_class.ImageStruct import ImageStruct
import heapq
import torch

In [2]:
annotation_file, tensor_file = \
      './data/anotation_new.csv', './image_tensor/model_image_2023-07-08/resnet50_tensor.pt'
images = ImageStruct(annotation_file, tensor_file)

In [3]:
src_index = random.randint(0, len(images))
s_img_tensor, s_img_path, s_category, s_caption = images.get(src_index)
s_category, s_caption

('ニット × 白（ホワイト）',
 ' リズミカルに描いたマルチボーダーがインパクトを放つニットプルオーバー。コットンアクリルをハイゲージで編み込み、季節らしい爽やかな風合いに仕上げました。デニムとの相性もよく、こなれ感たっぷりのカジュアルコーデが完成します。')

In [4]:
import torchvision.transforms as transforms
from PIL import Image
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn

if torch.cuda.is_available():
    device = torch.device("cuda")  # GPUデバイスを取得
else:
    device = torch.device("cpu")  # CPUデバイスを取得
class CaptionEncoder(nn.Module):
  def __init__(self):
    super().__init__()
    self.bert = AutoModel.from_pretrained("cl-tohoku/bert-base-japanese-v2")
  def forward(self, x):
    x = self.bert(x)
    x = x.last_hidden_state
    # print(x.shape)
    x = x[:,0,:] 
    # print(x.shape)
    return x

model_name =  './model/model_caption_2023-07-08.pth'
caption_model = CaptionEncoder().to(device)
caption_model.load_state_dict(torch.load(model_name))
caption_model.eval()

tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-v2")


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
input_text = "夏 薄着 暑い 白い 白"
ids = tokenizer.encode(input_text, return_tensors='pt')
ids = ids.to(device)
s_caption_tensor = caption_model(ids)[0].to('cpu')

In [13]:
from util.category import get_item_id

In [32]:

heap = []
MAX = 100000
for i in range(MAX):
    if (i % 1000 == 0):
        print(f"{i * 100 / MAX} %")
    # img_tensor, img_path, category
    t_img_tensor, t_img_path, t_category, _ = images.get(i)
    t_id = get_item_id(t_img_path)
    heap_ids = [get_item_id(i[1]) for i in heap]
    if (t_id in heap_ids):
        continue
    # print(s_category, t_category)
    # if (s_category != t_category):
    #     continue
    dist = torch.dist(s_caption_tensor, t_img_tensor, p=2)
    heapq.heappush(heap, (-dist.item(), t_img_path))
    if (len(heap) > 10):
        heapq.heappop(heap)
    
for i in range(len(heap)):
    print(heap[i])

0.0 %
1.0 %
2.0 %
3.0 %
4.0 %
5.0 %
6.0 %
7.0 %
8.0 %
9.0 %
10.0 %
11.0 %
12.0 %
13.0 %
14.0 %
15.0 %
16.0 %
17.0 %
18.0 %
19.0 %
20.0 %
21.0 %
22.0 %
23.0 %
24.0 %
25.0 %
26.0 %
27.0 %
28.0 %
29.0 %
30.0 %


In [ ]:
from util.save_combined_image import save_combined_image

from datetime import datetime
import os

# 現在の日付を取得します
now = datetime.now()

# YYYY-MM-DD形式で日付を出力します
formatted_date = now.strftime("%m-%d")
os.mkdir(f'./result/{formatted_date}/{input_text}')

save_combined_image(s_img_path, [i[1] for i in heap], f'./result/{formatted_date}/{input_text}/normal.png')